In [ ]:
!mkdir videos

In [ ]:
!pip install git+https://github.com/hukkelas/DSFD-Pytorch-Inference.git

In [1]:
import face_detection
import cv2
import numpy as np
import os

print(face_detection.available_detectors)

['DSFDDetector', 'RetinaNetResNet50', 'RetinaNetMobileNetV1']


In [ ]:
def anonymize_face_pixelate(image, blocks=3):
	# divide the input image into NxN blocks
	(h, w) = image.shape[:2]
	xSteps = np.linspace(0, w, blocks + 1, dtype="int")
	ySteps = np.linspace(0, h, blocks + 1, dtype="int")

	# loop over the blocks in both the x and y direction
	for i in range(1, len(ySteps)):
		for j in range(1, len(xSteps)):
			# compute the starting and ending (x, y)-coordinates
			# for the current block
			startX = xSteps[j - 1]
			startY = ySteps[i - 1]
			endX = xSteps[j]
			endY = ySteps[i]

			# extract the ROI using NumPy array slicing, compute the
			# mean of the ROI, and then draw a rectangle with the
			# mean RGB values over the ROI in the original image
			roi = image[startY:endY, startX:endX]
			(B, G, R) = [int(x) for x in cv2.mean(roi)[:3]]
			cv2.rectangle(image, (startX, startY), (endX, endY),
				(B, G, R), -1)

	# return the pixelated blurred image
	return image

In [ ]:
video_folder = '/content/videos/'

boxes = True 

filenames = []
filepaths = []
for i in sorted(os.listdir(video_folder)):
    if i[-4:] == '.mp4':
        filenames.append(i[:-4])
        filepaths.append(video_folder+i)
print(filenames,filepaths)

output_filepaths = []
for i in filenames:
    if boxes is True:
        output_filepaths.append(video_folder+i+'_boxes.avi')
    else:
        output_filepaths.append(video_folder+i+'_blurred.avi')
print(output_filepaths)

In [ ]:
for i,video in enumerate(filepaths):
    cap = cv2.VideoCapture(video)

    frame_width = int(cap.get(3)) 
    frame_height = int(cap.get(4)) 
    size = (frame_width, frame_height) 

    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out = cv2.VideoWriter(output_filepaths[i],fourcc, 20.0, size)

    detector = face_detection.build_detector(
    "DSFDDetector", confidence_threshold=.5, nms_iou_threshold=.3)

    while(cap.isOpened()):
        ret, frame = cap.read()
        if not ret:
            break
        detections = detector.detect(frame)
        
        for face in detections:
            if face[4] > 0.5:
                if boxes is True:
                    cv2.rectangle(frame,(face[0],face[1]),(face[2],face[3]),(255.0,0),3)
                else:
                    roi = frame[int(face[1]):int(face[3]),int(face[0]):int(face[2])]
                    roi = anonymize_face_pixelate(roi,blocks=4)
                    frame[int(face[1]):int(face[3]),int(face[0]):int(face[2])] = roi
                
        out.write(frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()